In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
% mv /Users/codetesting/Downloads/benchmark.zip benchmark.zip

mv: /Users/codetesting/Downloads/benchmark.zip: No such file or directory


In [3]:
summary = pd.read_csv("benchmark_data/index.csv")

In [4]:
hedder = pd.read_csv("benchmark_data/header.csv")

In [5]:
summary.columns = hedder.columns

In [6]:
summary.tail(5)

,lr,reward_lr,epochs,reward_epochs,k,ch_h,agent,num_iters,max_delay,hyperbolic_constant,algorithm,reward_prob,imsize,model,seeds,batchsize,use_gpu,final_accuracy,performance
1148,0.025,0.1,30,50,10,150,myopic,50,7,1.0,vi_inference,0.05,16,VI,2-3-5-8-13-21-34-55,20,True,None,0.9133453765302936
1149,0.025,0.1,30,50,10,150,myopic,50,7,1.0,vi_inference,0.05,16,VI,3-5-8-13-21-34-55-89,20,True,None,0.9050034539818632
1150,0.025,0.1,30,50,10,150,myopic,50,7,1.0,vi_inference,0.05,16,VI,1-2-3-4-5-6-7-8,20,True,None,0.9093241767884133
1151,0.025,0.1,30,50,10,150,sophisticated,50,5,0.9,given_rewards,0.05,8,SIMPLE,1-2-3-5-8-13-21-34,20,True,83.39921872317791,0.9912607749893483
1152,0.025,0.1,30,50,10,150,sophisticated,50,5,0.9,given_rewards,0.05,8,SIMPLE,2-3-5-8-13-21-34-55,20,True,82.94296878576279,0.9799702790324849


In [7]:
completed_frame = summary.loc[summary[' performance'] != ' None']
completed_frame = completed_frame.copy()

## Number of successful calls

Algorithms:

In [8]:
completed_frame[' algorithm'].value_counts()

 vi_inference         218
 given_rewards        129
 boltzmann_planner    128
 no_rewards           128
Name:  algorithm, dtype: int64

Models:

In [9]:
completed_frame[' model'].value_counts()

 VIN       384
 SIMPLE    123
 VI         96
Name:  model, dtype: int64

In [10]:
# This is not used
useful = [' algorithm', ' model', ' final_accuracy', ' performance']

In [11]:
# Create a column called 'perf' with the float values of the ' perfomance' column
compare_success = completed_frame.loc[:, (' performance')].apply(lambda x: float(x.split()[0]))
completed_frame['perf'] = compare_success

## Analysis: Useful functions

In [12]:
def get_rows_by_col(pdframe, colname, val):
    """Gets the rows where the colname meets some value"""
    return pdframe.loc[pdframe[colname]==val]

def get_nontrivial_columns(pdframe):
    """Gets the nontrivial columns from the pdframe"""
    nontrivial = []
    for column in pdframe.columns:
        if pdframe[column].unique().size > 1:
            nontrivial.append(column)
    return nontrivial

def get_trans_col(frame, grouped, func_name, colname):
    """Gets the row with the value given by grouped.transform(func_name)"""
    idx = grouped.transform(func_name) == frame[colname]
    return frame.loc[idx]

def print_basic_stats(pdframe, group_col, colname):
    """Prints basic statistics about grouped columns
    
    i.e. conditions underwhich max & min are achieved
    
    group_col –– the column we want to groupby (e.g. ' model')
    colname   –– the values we want to compare (e.g. 'perf')
    """
    pdgrouped = pdframe.groupby(group_col)[colname]
    max_row = get_trans_col(pdframe, pdgrouped, 'max', colname)
    min_row = get_trans_col(pdframe, pdgrouped, 'min', colname)
    
    print("\nMean performance of models is: ")
    print(pdgrouped.mean())
    print("\nStd performance of models is: ")
    print(pdgrouped.std())
    print("\nMax performance of models is: ")
    print(max_row)
    print("\nMin performance of models is: ")
    print(min_row)

## Analyzing: given_rewards

In [13]:
given_rewards = get_rows_by_col(completed_frame, ' algorithm', ' given_rewards')

In [14]:
# Get nontrivial columns of given_rewards, and compare by performance
cols = get_nontrivial_columns(given_rewards)
print_basic_stats(given_rewards[cols], ' model', 'perf')


Mean performance of models is: 
 model
 SIMPLE    0.511206
 VIN       0.668806
Name: perf, dtype: float64

Std performance of models is: 
 model
 SIMPLE    0.364155
 VIN       0.200970
Name: perf, dtype: float64

Max performance of models is: 
        agent   max_delay   hyperbolic_constant   imsize    model  \
6     optimal           5                   0.9        8      VIN   
287   optimal           7                   0.9        8   SIMPLE   

                   seeds      final_accuracy          performance      perf  
6        1-2-3-4-5-6-7-8   90.12187501043081   0.9243739500762154  0.924374  
287   1-2-3-5-8-13-21-34   83.39140617847443   0.9978897509131921  0.997890  

Min performance of models is: 
       agent   max_delay   hyperbolic_constant   imsize    model  \
577   myopic           5                   0.9        8   SIMPLE   
603   myopic           5                   0.9       16      VIN   

                     seeds      final_accuracy          performance      per

## Analyzing: no_rewards

In [15]:
no_rewards = get_rows_by_col(completed_frame, ' algorithm', ' no_rewards')

In [16]:
cols = get_nontrivial_columns(no_rewards)
print_basic_stats(no_rewards[cols], ' model', 'perf')


Mean performance of models is: 
 model
 SIMPLE    0.384627
 VIN       0.699094
Name: perf, dtype: float64

Std performance of models is: 
 model
 SIMPLE    0.355292
 VIN       0.217515
Name: perf, dtype: float64

Max performance of models is: 
        agent   max_delay   hyperbolic_constant   imsize    model  \
182   optimal           5                   1.0        8   SIMPLE   
762    myopic           5                   1.0        8      VIN   

                seeds      final_accuracy          performance      perf  
182   1-2-3-4-5-6-7-8   82.89765620231628   0.9976912485982808  0.997691  
762   1-2-3-4-5-6-7-8   88.30468744039536   0.9853662344351659  0.985366  

Min performance of models is: 
       agent   max_delay   hyperbolic_constant   imsize    model  \
755   myopic           5                   1.0        8   SIMPLE   
772   myopic           5                   1.0       12      VIN   

                    seeds      final_accuracy           performance      perf  
755  

## Analyzing: boltzmann_planner

In [17]:
b_planner = get_rows_by_col(completed_frame, ' algorithm', ' boltzmann_planner')

In [18]:
cols = get_nontrivial_columns(b_planner)
print_basic_stats(b_planner[cols], ' model', 'perf')


Mean performance of models is: 
 model
 SIMPLE    0.569295
 VIN       0.684930
Name: perf, dtype: float64

Std performance of models is: 
 model
 SIMPLE    0.338840
 VIN       0.231216
Name: perf, dtype: float64

Max performance of models is: 
        agent   max_delay   hyperbolic_constant   imsize    model  \
73    optimal           5                   0.9        8   SIMPLE   
507   optimal           7                   1.0        8      VIN   

                     seeds      final_accuracy          performance      perf  
73    3-5-8-13-21-34-55-89   83.43671880662441   0.9978364895812957  0.997836  
507     1-2-3-5-8-13-21-34   89.67890626192093   0.9419282325215252  0.941928  

Min performance of models is: 
       agent   max_delay   hyperbolic_constant   imsize    model  \
648   myopic           5                   0.9        8   SIMPLE   
822   myopic           5                   1.0       16      VIN   

                    seeds      final_accuracy           performance   

## Analyzing: vi_inference

In [19]:
vi_alg = get_rows_by_col(completed_frame, ' algorithm', ' vi_inference')
cols = get_nontrivial_columns(vi_alg)
print_basic_stats(vi_alg[cols], ' model', 'perf')


Mean performance of models is: 
 model
 SIMPLE    0.080139
 VI        0.923487
 VIN       0.023981
Name: perf, dtype: float64

Std performance of models is: 
 model
 SIMPLE    0.136985
 VI        0.065232
 VIN       0.016410
Name: perf, dtype: float64

Max performance of models is: 
        agent   max_delay   hyperbolic_constant   imsize    model  \
112   optimal           5                   0.9        8      VIN   
253   optimal           5                   1.0        8   SIMPLE   
715    myopic           5                   0.9       16       VI   
718    myopic           5                   0.9       16       VI   
859    myopic           5                   1.0       16       VI   

                     seeds           performance      perf  
112    2-3-5-8-13-21-34-55   0.06828133088948023  0.068281  
253   3-5-8-13-21-34-55-89    0.7335380404446011  0.733538  
715     1-2-3-5-8-13-21-34    0.9921396771749191  0.992140  
718        1-2-3-4-5-6-7-8    0.9921396771749191  0.9921